# GabeDA Synthetic Features - Auto-Inference

This notebook demonstrates **automatic column inference** using synthetic features in the GabeDA preprocessing pipeline.

## What This Notebook Does

1. **Defines Inference Functions** - Creates functions to calculate missing columns from available data:
   - Cost calculations (`in_cost_unit`, `in_cost_total`)
   - Price calculations (`in_price_unit`, `in_price_total`)
   - Discount calculations (`in_discount_total` from gross or rate)
   - Commission calculations (`in_commission_total` from rate or margin)
   - Margin calculations (`in_margin` with optional adjustments)

2. **Registers Features** - Stores inference functions in the feature store (`feature_store/synthetic/`)

3. **Auto-Enrichment** - Shows the "lucky case" detection system:
   - Loads incomplete test data (e.g., `lucky_01_cost_total.csv` with only 6 columns)
   - Automatically detects which columns can be inferred
   - Calculates missing columns using stored synthetic features
   - Returns enriched dataset with all inferable columns added

## The "Lucky Case" System

The enricher analyzes input data and calculates a **"luck score"** - how many missing columns can be automatically inferred based on available data. For example:

- **Input**: 6 columns (`in_dt`, `in_trans_id`, `in_product_id`, `in_quantity`, `in_cost_total`, `in_price_total`)
- **Output**: 10 columns (4 auto-inferred: `in_cost_unit`, `in_price_unit`, `in_discount_total`, `in_margin`)

## Key Components

- **SyntheticEnricher** ([src/preprocessing/synthetic.py](src/preprocessing/synthetic.py)) - Auto-inference engine
- **Test Cases** ([data/tests/synthetic/](data/tests/synthetic/)) - Sample datasets with varying completeness
- **Base Case Schema** ([data/tests/synthetic/base_case.csv](data/tests/synthetic/base_case.csv)) - Defines which columns are inferable

Try different test cases to see how the system adapts to different input scenarios!

## 1. Configuration

## 0. Project Root Setup (Auto-generated)

In [ ]:
# Auto-detect project root and add to Python path
import os
import sys
from pathlib import Path

# Get the project root (2 levels up from notebooks/development or notebooks/from_store)
notebook_dir = Path.cwd() if '__file__' not in globals() else Path(__file__).parent
project_root = notebook_dir.parent.parent

# Change to project root
os.chdir(project_root)

# Add project root to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Working directory: {os.getcwd()}")
print(f"Project root: {project_root}")

In [20]:
# Base configuration
base_cfg = {
    # Try one of the lucky test cases to see auto-inference in action!
    # 'input_file': 'data/tests/synthetic/lucky_01_cost_total.csv',  # Best: 5 columns can be inferred!
    'input_file': 'data/tests/raw/quick_test_7days.csv',
    # 'input_file': 'data/tests/synthetic/lucky_02_cost_unit.csv',  #  5 inferable columns
    # 'input_file': 'data/tests/synthetic/lucky_04_cost_discount.csv',  # 4 inferable columns
    # 'input_file': 'data/tests/synthetic/tc_02_complete_all.csv',  # Complete: No inference needed
    
    'client': '00_synthetic',
    'analysis_dt': '2025-11-11',
    'log_level': 'INFO',
    'fidx_config': {'type': 'local', 'path': 'feature_store'},
    
    # Default formats applied to all columns of each dtype (unless overridden)s
    'default_formats': {
        'date': '%d/%m/%Y %H:%M',  # European date format
        'float': {'thousands': '.', 'decimal': ','},  # European numeric format
        'int': {'thousands': '.', 'decimal': ','},
    },
    
    # Data schema: column mapping + types
    'data_schema': {
        'in_dt': {
            'source_column': 'in_dt',
            'dtype': 'date',
            # 'format': '%d-%m-%Y %H:%M'
        },
        'in_trans_id': {
            'source_column': 'in_trans_id',
            'dtype': 'str'
        },
        'in_trans_type': {
            'source_column': 'in_trans_type',
            'dtype': 'str'
        },
        'in_customer_id': {
            'source_column': 'in_customer_id',
            'dtype': 'str'
        },
        'in_product_id': {
            'source_column': 'in_product_id',
            'dtype': 'str'
        },
        'in_description': {
            'source_column': 'in_description',
            'dtype': 'str'
        },
        'in_category': {
            'source_column': 'in_category',
            'dtype': 'str'
        },
        'in_unit_type': {
            'source_column': 'in_unit_type',
            'dtype': 'str'
        },
        'in_stock': {
            'source_column': 'in_stock',
            'dtype': 'float'
        },
        'in_quantity': {
            'source_column': 'in_quantity',
            'dtype': 'float'
        },
        'in_cost_unit': {
            'source_column': 'in_cost_unit',
            'dtype': 'float'
        },
        'in_cost_total': {
            'source_column': 'in_cost_total',
            'dtype': 'float'
        },
        'in_price_unit': {
            'source_column': 'in_price_unit',
            'dtype': 'float'
        },
        'in_price_total': {
            'source_column': 'in_price_total',
            'dtype': 'float'
        },
        'in_discount_total': {
            'source_column': 'in_discount_total',
            'dtype': 'float'
        },
        'in_commission_total': {
            'source_column': 'in_commission_total',
            'dtype': 'float'
        },
        'in_margin': {
            'source_column': 'in_margin',
            'dtype': 'float'
        },
    }
}

## 2. Imports (v2.0 Refactored)

Notice the new modular imports from the 6 packages:

In [21]:
import pandas as pd
import numpy as np

# v2.0 Refactored imports
from src.utils.logger import setup_logging, get_logger
from src.core.context import GabedaContext
from src.preprocessing.loaders import DataLoader
from src.preprocessing.validators import DataValidator
from src.preprocessing.schema import SchemaProcessor
from src.preprocessing.synthetic import SyntheticEnricher  # NEW: Synthetic enrichment
from src.features.store import FeatureStore
from src.features.resolver import DependencyResolver
from src.features.detector import FeatureTypeDetector
from src.features.analyzer import FeatureAnalyzer
from src.execution.calculator import FeatureCalculator
from src.execution.groupby import GroupByProcessor
from src.execution.executor import ModelExecutor
from src.export.excel import ExcelExporter

print("[OK] All imports successful!")

[OK] All imports successful!


## 3. Initialize Context & Logging

In [22]:
# Setup logging
setup_logging(log_level=base_cfg.get('log_level', 'INFO'), config=base_cfg)
logger = get_logger(__name__)

# Initialize context
ctx = GabedaContext(base_cfg)

print(f"[OK] Context initialized: {ctx.run_id}")

📝 Run instance ID: 00_synthetic_20251020_135128 - Logging [INFO] to: logs\00_synthetic_20251020_135128.log
[OK] Context initialized: 00_synthetic_20251020_135128


## 4. Load & Preprocess Data

Using the new modular preprocessing components:

### 4.1 Load

In [23]:
# Load raw data
loader = DataLoader()
raw_data = loader.load_csv(base_cfg['input_file'])
print(f"[OK] Loaded raw data: {raw_data.shape}")

[OK] Loaded raw data: (138, 17)


### 4.2 Validation

In [24]:
# Validate required columns
validator = DataValidator()
required_cols = [spec['source_column'] for spec in base_cfg['data_schema'].values()]
validation = validator.validate_required_columns(raw_data, required_cols)

if validation.is_valid:
    print(f"[OK] Validation passed")
else:
    print(f"[X] Validation failed: {validation.errors}")


[OK] Validation passed


In [25]:
# OPTIONAL Validate data quality
quality_validation = validator.validate_data_quality(raw_data, base_cfg['data_schema'])
if quality_validation.is_valid:
    print(f"[OK] Quality validation passed")
else:
    print(f"[X] Quality validation issues: {quality_validation.issues}")

[OK] Quality validation passed


### 4.3 Schema

In [26]:
# Process schema (column mapping + type conversion)
schema_processor = SchemaProcessor()
result = schema_processor.process_schema(raw_data, base_cfg)
preprocessed_df = result.df

print(f"[OK] Preprocessed data: {preprocessed_df.shape}")
print(f"[OK] Available columns: {result.available_cols}")

[OK] Preprocessed data: (138, 17)
[OK] Available columns: ['in_dt', 'in_trans_id', 'in_trans_type', 'in_customer_id', 'in_product_id', 'in_description', 'in_category', 'in_unit_type', 'in_stock', 'in_quantity', 'in_cost_unit', 'in_cost_total', 'in_price_unit', 'in_price_total', 'in_discount_total', 'in_commission_total', 'in_margin']


### 4.4 Synthetic Enrichment (Auto-Inference)

**AUTOMATIC COLUMN INFERENCE**: After schema processing, check if we can infer missing columns using synthetic features.

This is the "lucky" case detection - if the input data happens to have the right combination of columns, we can automatically calculate missing columns.

In [27]:
# Initialize enricher (loads features from feature_store/synthetic/)
enricher = SyntheticEnricher(synthetic_model_name='synthetic')

# Check "luck score" - how many columns can be inferred?
luck_info = enricher.get_lucky_score(preprocessed_df)

print(f"\n{'='*60}")
print(f"LUCK ANALYSIS")
print(f"{'='*60}")
print(f"Input columns: {preprocessed_df.shape[1]}")
print(f"Missing inferable columns: {luck_info['total_possible']}")
print(f"Can infer: {luck_info['luck_score']} columns ({luck_info['luck_percentage']:.1f}%)")

if luck_info['luck_score'] > 0:
    print(f"\n✨ LUCKY! Can infer these columns:")
    for i, col in enumerate(luck_info['missing_columns'], 1):
        feat = luck_info['applicable_features'][i-1] if i-1 < len(luck_info['applicable_features']) else '?'
        print(f"   {i}. {col} (using: {feat})")
else:
    print(f"\n⚠ No columns can be inferred from current data")

print(f"{'='*60}\n")


LUCK ANALYSIS
Input columns: 17
Missing inferable columns: 0
Can infer: 0 columns (100.0%)

⚠ No columns can be inferred from current data



In [28]:
# Apply synthetic enrichment if lucky
if luck_info['luck_score'] > 0:
    print("Applying synthetic enrichment...")
    
    enriched_df = enricher.enrich(data=preprocessed_df)
    
    # Show what was added
    new_cols = [col for col in enriched_df.columns if col not in preprocessed_df.columns]
    
    print(f"\n[OK] Enrichment complete!")
    print(f"   Before: {preprocessed_df.shape[1]} columns")
    print(f"   After: {enriched_df.shape[1]} columns")
    print(f"   Added: {len(new_cols)} columns - {new_cols}")
    
    # Update preprocessed_df with enriched version
    preprocessed_df = enriched_df
    
    print(f"\n✅ preprocessed_df updated with {len(new_cols)} new columns!")
else:
    print("\n⚠ No enrichment applied - keeping original data")


⚠ No enrichment applied - keeping original data


# Store datasets in context (after potential enrichment)
ctx.set_dataset('transactions_raw', raw_data)
ctx.set_dataset('transactions_enriched', preprocessed_df)  # This may have been enriched
ctx.list_datasets()

In [29]:
# Store datasets in context
ctx.set_dataset('transactions_raw', raw_data)
ctx.set_dataset('transactions_enriched', preprocessed_df)
ctx.list_datasets()

['transactions_raw', 'transactions_enriched']

In [30]:
preprocessed_df.head()

,in_dt,in_trans_id,in_product_id,in_quantity,in_price_total,in_trans_type,in_customer_id,in_description,in_category,in_unit_type,in_stock,in_cost_unit,in_cost_total,in_price_unit,in_discount_total,in_commission_total,in_margin
0,2025-01-10 00:59:00,trans000002,prod5,4.0,73933.0,sale,client4,product 5,category C,unit,50.0,11308.0,45231.0,18483.0,0.0,2562.0,26139.0
1,2025-01-10 09:55:00,trans000014,prod10,5.0,201717.0,sale,client5,product 10,category A,box,69.0,26164.0,130822.0,40343.0,0.0,6623.0,64272.0
2,2025-01-10 09:59:00,trans000009,prod4,4.0,131738.0,sale,client9,product 4,category B,unit,35.0,19198.0,76791.0,32934.0,0.0,6616.0,48331.0
3,2025-01-10 10:17:00,trans000019,prod4,4.0,119824.0,sale,client13,product 4,category B,unit,144.0,18972.0,75889.0,29956.0,0.0,5963.0,37972.0
4,2025-01-10 11:00:00,trans000017,prod7,3.0,99428.0,return,client6,product 7,category A,unit,53.0,24769.0,74308.0,33143.0,0.0,5394.0,19725.0


In [31]:
preprocessed_df.columns

Index(['in_dt', 'in_trans_id', 'in_product_id', 'in_quantity',
       'in_price_total', 'in_trans_type', 'in_customer_id', 'in_description',
       'in_category', 'in_unit_type', 'in_stock', 'in_cost_unit',
       'in_cost_total', 'in_price_unit', 'in_discount_total',
       'in_commission_total', 'in_margin'],
      dtype='object')

In [32]:
# preprocessed_df.to_csv('data/test_client/preprocessed_df.csv', index=False)

## 5. Define Features

### Filters (row-level calculations)

### Inference Strategy Reference

Based on [base_case.csv](data/tests/synthetic/base_case.csv), the following columns are **inferable** (can be calculated from other columns):

| Inferable Column | Inference Options | Required Input Columns |
|-----------------|-------------------|------------------------|
| **in_cost_unit** | `in_cost_unit()` | `in_cost_total`, `in_quantity` |
| **in_cost_total** | `in_cost_total()` | `in_cost_unit`, `in_quantity` |
| **in_price_unit** | `in_price_unit()` | `in_price_total`, `in_quantity` |
| **in_price_total** | `in_price_total()` | `in_price_unit`, `in_quantity` |
| **in_discount_total** | `in_discount_total_from_gross()` | `in_price_unit`, `in_quantity`, `in_price_total` |
| | `in_discount_total_from_rate()` | `in_price_total`, `in_discount_rate` |
| **in_commission_total** | `in_commission_total_from_rate()` | `in_price_total`, `in_commission_rate` |
| | `in_commission_total_from_margin()` | `in_margin`, `in_commission_rate` |
| **in_margin** | `in_margin_from_totals()` | `in_price_total`, `in_cost_total` |
| | `in_margin_from_units()` | `in_price_unit`, `in_cost_unit`, `in_quantity` |
| | `in_margin_with_discounts()` | `in_price_total`, `in_cost_total`, `in_discount_total`, `in_commission_total` |
| | `in_margin()` (recommended) | `in_price_total`, `in_cost_total`, [optional: `in_discount_total`, `in_commission_total`] |

**Note**: 
- Multiple functions are provided for columns that can be inferred in different ways
- Choose the appropriate function based on which input columns are available
- The dependency resolver will automatically determine the correct execution order

In [33]:
"""
Inferable Column Functions

Based on base_case.csv, these columns can be inferred from other columns:
1. in_cost_unit - from in_cost_total / in_quantity
2. in_cost_total - from in_cost_unit * in_quantity
3. in_price_unit - from in_price_total / in_quantity
4. in_price_total - from in_price_unit * in_quantity
5. in_discount_total - from pricing differences
6. in_commission_total - from business rules
7. in_margin - from price - cost calculations
"""

# ============================================================================
# COST CALCULATIONS
# ============================================================================

def in_cost_unit(in_cost_total, in_quantity):
    """Calculate unit cost from total cost and quantity."""
    return in_cost_total / in_quantity if in_quantity != 0 else 0.0

def in_cost_total(in_cost_unit, in_quantity):
    """Calculate total cost from unit cost and quantity."""
    return in_cost_unit * in_quantity


# ============================================================================
# PRICE CALCULATIONS
# ============================================================================

def in_price_unit(in_price_total, in_quantity):
    """Calculate unit price from total price and quantity."""
    return in_price_total / in_quantity if in_quantity != 0 else 0.0

# def in_price_total(in_price_unit, in_quantity):
#     """Calculate total price from unit price and quantity."""
#     return in_price_unit * in_quantity


# ============================================================================
# DISCOUNT CALCULATIONS
# ============================================================================

def in_discount_total_from_gross(in_price_unit, in_quantity, in_price_total):
    """
    Calculate discount from the difference between gross and net price.
    Assumes in_price_total is net (after discount).
    """
    gross_total = in_price_unit * in_quantity
    return gross_total - in_price_total if gross_total > in_price_total else 0.0

def in_discount_total_from_rate(in_price_total, in_discount_rate):
    """Calculate discount from price and discount rate (e.g., 0.10 for 10%)."""
    # Note: This assumes in_price_total is BEFORE discount
    return in_price_total * in_discount_rate


# ============================================================================
# COMMISSION CALCULATIONS
# ============================================================================

def in_commission_total_from_rate(in_price_total, in_commission_rate):
    """Calculate commission from total price and commission rate."""
    return in_price_total * in_commission_rate

def in_commission_total_from_margin(in_margin, in_commission_rate):
    """Calculate commission from margin and commission rate."""
    return in_margin * in_commission_rate


# ============================================================================
# MARGIN CALCULATIONS
# ============================================================================

def in_margin_from_totals(in_price_total, in_cost_total):
    """Calculate margin from total price minus total cost."""
    return in_price_total - in_cost_total

def in_margin_from_units(in_price_unit, in_cost_unit, in_quantity):
    """Calculate margin from unit prices and quantity."""
    return (in_price_unit - in_cost_unit) * in_quantity

def in_margin_with_discounts(in_price_total, in_cost_total, in_discount_total, in_commission_total):
    """Calculate net margin considering discounts and commissions."""
    return in_price_total - in_cost_total - in_discount_total - in_commission_total


# ============================================================================
# ALTERNATIVE: Combined margin calculation (most complete)
# ============================================================================

def in_margin(in_price_total, in_cost_total, in_discount_total=0.0, in_commission_total=0.0):
    """
    Calculate margin with optional discount and commission adjustments.
    
    Margin = Revenue - Cost - Discount - Commission
    
    Note: Parameters with default values can be omitted from input if not available.
    """
    margin = in_price_total - in_cost_total
    if in_discount_total:
        margin -= in_discount_total
    if in_commission_total:
        margin -= in_commission_total
    return margin

### Attributes (aggregated calculations)

In [34]:
None

## 6. Configure Model

### Define features

In [35]:
features = {
    'in_cost_unit': in_cost_unit,
    'in_cost_total': in_cost_total,
    'in_price_unit': in_price_unit,
    'in_discount_total_from_gross': in_discount_total_from_gross,
    'in_discount_total_from_rate': in_discount_total_from_rate,
    'in_commission_total_from_rate': in_commission_total_from_rate,
    'in_commission_total_from_margin': in_commission_total_from_margin,
    'in_margin_from_totals': in_margin_from_totals,
    'in_margin_from_units': in_margin_from_units,
    'in_margin_with_discounts': in_margin_with_discounts,
    'in_margin': in_margin,  # Most complete version
}

### Define Model Configuration

In [36]:
# Model configuration
cfg_product = {
    'model_name': 'synthetic',
    # 'group_by': ['in_product_id'],
    # 'group_by': ['in_product_id','in_dt'],
    'row_id': 'in_trans_id',
    'output_cols': [
        # Filters
        'in_cost_unit',
        'in_cost_total',
        'in_price_unit',
    ],
    'features': features
}

print("[OK] Model configured")
print(f"  - Model: {cfg_product['model_name']}")
if cfg_product.get('group_by'): print(f"  - Group by: {cfg_product['group_by']}")
print(f"  - Output columns: {len(cfg_product['output_cols'])}")

[OK] Model configured
  - Model: synthetic
  - Output columns: 3


### Initialize Enricher with Feature Store

Now that features are stored, initialize the enricher with the same feature store:

### Feature Store

In [37]:
# Initialize feature store
feature_store = FeatureStore()

# New method to store multiple features at once
feature_store.store_features(features, model_name = cfg_product['model_name'], auto_save=True)

print(f"[OK] Stored {len(feature_store.features)} features in store")
# Features stored successfully

[OK] Stored 11 features in store


## 7. Resolve Dependencies

The DependencyResolver uses DFS traversal to determine execution order:

In [38]:
# Initialize resolver
resolver = DependencyResolver(feature_store)

# Resolve dependencies
# FIXED: Pass group_by directly (not wrapped in a list)
in_cols, exec_seq, ext_cols = resolver.resolve_dependencies(
    output_cols=cfg_product['output_cols'],
    available_cols=preprocessed_df.columns.tolist(),
    group_by=cfg_product.get('group_by'),  # Pass directly or None
    model=cfg_product['model_name']
)

# Update config
cfg_product['in_cols'] = in_cols
cfg_product['exec_seq'] = exec_seq
cfg_product['ext_cols'] = ext_cols

print("[OK] Dependencies resolved")
print(f"  - Input columns needed: {in_cols}")
print(f"  - Execution sequence: {exec_seq}")
if ext_cols:
    print(f"  - External columns requested:")
    for ext_name, cols in ext_cols.items():
        print(f"    - {ext_name}: {cols}")

[OK] Dependencies resolved
  - Input columns needed: ['in_cost_unit', 'in_cost_total', 'in_price_unit']
  - Execution sequence: []


### Save Master Configuration

Save the master_cfg.json file containing model metadata (model_name, model_seq, group_by, row_id):

In [39]:
# Save master_cfg.json
feature_store.save_master_config(
    model_name=cfg_product['model_name'],
    model_config=cfg_product
)

print(f"[OK] master_cfg.json saved for model '{cfg_product['model_name']}'")
print(f"  Location: feature_store/{cfg_product['model_name']}/master_cfg.json")

[OK] master_cfg.json saved for model 'synthetic'
  Location: feature_store/synthetic/master_cfg.json


## 8. Execute Model

Initialize execution components and run the model:

In [40]:
# Initialize execution components
detector = FeatureTypeDetector()
analyzer = FeatureAnalyzer(feature_store, detector)
calculator = FeatureCalculator()
groupby_processor = GroupByProcessor(calculator, detector)
executor = ModelExecutor(analyzer, groupby_processor)

print("[OK] Execution pipeline initialized")

[OK] Execution pipeline initialized


In [41]:
# Execute model
output = executor.execute_model(
    data_in=preprocessed_df,
    cfg_model=cfg_product,
    input_dataset_name='transactions_enriched'
)

# Store results in context
ctx.set_model_output(cfg_product['model_name'], output, cfg_product)

print("\n[OK] Model executed successfully!")
print(f"  - Filters: {output['filters'].shape if output['filters'] is not None else 'None'}")
print(f"  - Attributes: {output['attrs'].shape if output['attrs'] is not None else 'None'}")
print(f"  - Filter columns: {output['exec_fltrs']}")
print(f"  - Attribute columns: {output['exec_attrs']}")


[OK] Model executed successfully!
  - Filters: (138, 17)
  - Attributes: None
  - Filter columns: ['in_dt', 'in_product_id', 'in_quantity', 'in_price_total', 'in_trans_type', 'in_customer_id', 'in_description', 'in_category', 'in_unit_type', 'in_stock', 'in_cost_unit', 'in_cost_total', 'in_price_unit', 'in_discount_total', 'in_commission_total', 'in_margin']
  - Attribute columns: []


## 9. View Results

In [42]:
print("Available Datasets:")
for name in ctx.list_datasets():
    df = ctx.get_dataset(name)
    print(f"  - {name}: {df.shape}")

Available Datasets:
  - transactions_raw: (138, 17)
  - transactions_enriched: (138, 17)
  - synthetic_filters: (138, 17)


### Raw Data

In [43]:
ctx.get_dataset('transactions_raw').head()

,in_dt,in_trans_id,in_product_id,in_quantity,in_price_total,in_trans_type,in_customer_id,in_description,in_category,in_unit_type,in_stock,in_cost_unit,in_cost_total,in_price_unit,in_discount_total,in_commission_total,in_margin
0,10/01/2025 00:59,trans000002,prod5,4,739.33,sale,client4,product 5,category C,unit,50,113.08,452.31,184.83,0.0,25.62,261.39
1,10/01/2025 09:55,trans000014,prod10,5,2017.17,sale,client5,product 10,category A,box,69,261.64,1308.22,403.43,0.0,66.23,642.72
2,10/01/2025 09:59,trans000009,prod4,4,1317.38,sale,client9,product 4,category B,unit,35,191.98,767.91,329.34,0.0,66.16,483.31
3,10/01/2025 10:17,trans000019,prod4,4,1198.24,sale,client13,product 4,category B,unit,144,189.72,758.89,299.56,0.0,59.63,379.72
4,10/01/2025 11:00,trans000017,prod7,3,994.28,return,client6,product 7,category A,unit,53,247.69,743.08,331.43,0.0,53.94,197.25


### Preprocessed Data

In [44]:
ctx.get_dataset('transactions_enriched').head()

,in_dt,in_trans_id,in_product_id,in_quantity,in_price_total,in_trans_type,in_customer_id,in_description,in_category,in_unit_type,in_stock,in_cost_unit,in_cost_total,in_price_unit,in_discount_total,in_commission_total,in_margin
0,2025-01-10 00:59:00,trans000002,prod5,4.0,73933.0,sale,client4,product 5,category C,unit,50.0,11308.0,45231.0,18483.0,0.0,2562.0,26139.0
1,2025-01-10 09:55:00,trans000014,prod10,5.0,201717.0,sale,client5,product 10,category A,box,69.0,26164.0,130822.0,40343.0,0.0,6623.0,64272.0
2,2025-01-10 09:59:00,trans000009,prod4,4.0,131738.0,sale,client9,product 4,category B,unit,35.0,19198.0,76791.0,32934.0,0.0,6616.0,48331.0
3,2025-01-10 10:17:00,trans000019,prod4,4.0,119824.0,sale,client13,product 4,category B,unit,144.0,18972.0,75889.0,29956.0,0.0,5963.0,37972.0
4,2025-01-10 11:00:00,trans000017,prod7,3.0,99428.0,return,client6,product 7,category A,unit,53.0,24769.0,74308.0,33143.0,0.0,5394.0,19725.0


### Filters (Row-Level Results)

In [45]:
ctx.get_model_filters('synthetic').head()

,in_dt,in_trans_id,in_product_id,in_quantity,in_price_total,in_trans_type,in_customer_id,in_description,in_category,in_unit_type,in_stock,in_cost_unit,in_cost_total,in_price_unit,in_discount_total,in_commission_total,in_margin
0,2025-01-10 00:59:00,trans000002,prod5,4.0,73933.0,sale,client4,product 5,category C,unit,50.0,11308.0,45231.0,18483.0,0.0,2562.0,26139.0
1,2025-01-10 09:55:00,trans000014,prod10,5.0,201717.0,sale,client5,product 10,category A,box,69.0,26164.0,130822.0,40343.0,0.0,6623.0,64272.0
2,2025-01-10 09:59:00,trans000009,prod4,4.0,131738.0,sale,client9,product 4,category B,unit,35.0,19198.0,76791.0,32934.0,0.0,6616.0,48331.0
3,2025-01-10 10:17:00,trans000019,prod4,4.0,119824.0,sale,client13,product 4,category B,unit,144.0,18972.0,75889.0,29956.0,0.0,5963.0,37972.0
4,2025-01-10 11:00:00,trans000017,prod7,3.0,99428.0,return,client6,product 7,category A,unit,53.0,24769.0,74308.0,33143.0,0.0,5394.0,19725.0


### Attributes (Aggregated Results)

In [46]:
ctx.get_model_attrs('synthetic')

## 10. Export to Excel

In [47]:
# Initialize exporter
exporter = ExcelExporter(ctx)

# Export model
output_file = f'outputs/{cfg_product["model_name"]}_export.xlsx'
exporter.export_model(cfg_product['model_name'], output_file, include_input=True)

print(f"\n[SUCCESS] Export complete!")
print(f"File saved: {output_file}")
print("\nExpected Excel Structure:")
print("  Tab 1: transactions_enriched (input dataset)")
print(f"  Tab 2: {cfg_product['model_name']}_filters")
print(f"  Tab 3: {cfg_product['model_name']}_attrs")


[SUCCESS] Export complete!
File saved: outputs/synthetic_export.xlsx

Expected Excel Structure:
  Tab 1: transactions_enriched (input dataset)
  Tab 2: synthetic_filters
  Tab 3: synthetic_attrs
